### Imports

In [1]:
import math
import numpy as np
import pandas as pd
import pygeohash as gh

### Format Divvy geographical data 

Potentially superfluous: Dataframe doesn't seem impacted (initial geodata format might be the proper one)

In [2]:
# def transform_lonlat_features(X: pd.DataFrame) -> pd.DataFrame:

#     assert isinstance(X, pd.DataFrame)
#     lonlat_features = ["start_lat", "start_lon", "end_lat", "end_lon"]

#     def distances_vectorized(df: pd.DataFrame, start_lat: str, start_lon: str, end_lat: str, end_lon: str) -> dict:
#         """
#         Calculate the haverzine and manhattan distance between two points on the earth (specified in decimal degrees).
#         Vectorized version for pandas df
#         Computes distance in kms
#         """
#         earth_radius = 6371

#         lat_1_rad, lon_1_rad = np.radians(df[start_lat]), np.radians(df[start_lon])
#         lat_2_rad, lon_2_rad = np.radians(df[end_lat]), np.radians(df[end_lon])

#         dlon_rad = lon_2_rad - lon_1_rad
#         dlat_rad = lat_2_rad - lat_1_rad

#         manhattan_rad = np.abs(dlon_rad) + np.abs(dlat_rad)
#         manhattan_km = manhattan_rad * earth_radius


### Define Geohash Rides

In [3]:
def compute_geohash_rides(X: pd.DataFrame, precision: int = 5) -> np.ndarray:
    """
    Add a geohash (ex: "dr5rx") of len "precision" = 5 by default
    corresponding to each (lon,lat) tuple, for pick-up, and drop-off
    """
    assert isinstance(X, pd.DataFrame)

    X["geohash_start"] = X.apply(lambda x: gh.encode(
        x.start_lat, x.start_lon, precision=precision),
                                    axis=1)
    X["geohash_end"] = X.apply(lambda x: gh.encode(
        x.end_lat, x.end_lon, precision=precision),
                                    axis=1)
    return X #[["geohash_start", "geohash_end"]]

### Define Geohash Stations


In [4]:
from ml_logic.data_import import get_station_data

def compute_geohash_stations(X: pd.DataFrame, precision: int = 5) -> np.ndarray:
    """
    Add a geohash (ex: "dr5rx") of len "precision" = 5 by default
    corresponding to each (lon,lat) tuple, for pick-up, and drop-off
    """
    assert isinstance(X, pd.DataFrame)

    X["geohash"] = X.apply(lambda x: gh.encode(
        x.lat, x.lon, precision=precision),
                                    axis=1)
    return X #[["geohash_start", "geohash_end"]]

ModuleNotFoundError: No module named 'ml_logic'

In [5]:
from ml_logic.data_import import get_station_data

def compute_geohash_stations(precision: int = 5) -> np.ndarray:
    """
    Add a geohash (ex: "dr5rx") of len "precision" = 5 by default
    corresponding to each (lon,lat) tuple, for pick-up, and drop-off
    """
    df_stations=get_station_data()
    assert isinstance(df_stations, pd.DataFrame)

    df_stations["geohash"] = df_stations.apply(lambda x: gh.encode(
        x.lat, x.lon, precision=precision),
                                    axis=1)
    return df_stations 

ModuleNotFoundError: No module named 'ml_logic'

### Import and format Divvy data

In [5]:
path = '/Users/joachimclodic/code/G-Dolle/DIVVY_BIKE/raw_data/2021/Divvy_Trips_2021_Q1.csv'
df=pd.read_csv(path)
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,E19E6F1B8D4C42ED,electric_bike,2021-01-23 16:14:19,2021-01-23 16:24:44,California Ave & Cortez St,17660,NaN,NaN,41.900341,-87.696743,41.89,-87.72,member
1,DC88F20C2C55F27F,electric_bike,2021-01-27 18:43:08,2021-01-27 18:47:12,California Ave & Cortez St,17660,NaN,NaN,41.900333,-87.696707,41.90,-87.69,member
2,EC45C94683FE3F27,electric_bike,2021-01-21 22:35:54,2021-01-21 22:37:14,California Ave & Cortez St,17660,NaN,NaN,41.900313,-87.696643,41.90,-87.70,member
3,4FA453A75AE377DB,electric_bike,2021-01-07 13:31:13,2021-01-07 13:42:55,California Ave & Cortez St,17660,NaN,NaN,41.900399,-87.696662,41.92,-87.69,member
4,BE5E8EB4E7263A0B,electric_bike,2021-01-23 02:24:02,2021-01-23 02:24:45,California Ave & Cortez St,17660,NaN,NaN,41.900326,-87.696697,41.90,-87.70,casual


In [6]:
df_geo=df[['ride_id','start_lat','start_lng','end_lat','end_lng']]
df_geo.head()

,ride_id,start_lat,start_lng,end_lat,end_lng
0,E19E6F1B8D4C42ED,41.900341,-87.696743,41.89,-87.72
1,DC88F20C2C55F27F,41.900333,-87.696707,41.90,-87.69
2,EC45C94683FE3F27,41.900313,-87.696643,41.90,-87.70
3,4FA453A75AE377DB,41.900399,-87.696662,41.92,-87.69
4,BE5E8EB4E7263A0B,41.900326,-87.696697,41.90,-87.70


In [7]:
df_geo=df_geo.rename(columns={'start_lng':'start_lon','end_lng':'end_lon'})
df_geo.head()

,ride_id,start_lat,start_lon,end_lat,end_lon
0,E19E6F1B8D4C42ED,41.900341,-87.696743,41.89,-87.72
1,DC88F20C2C55F27F,41.900333,-87.696707,41.90,-87.69
2,EC45C94683FE3F27,41.900313,-87.696643,41.90,-87.70
3,4FA453A75AE377DB,41.900399,-87.696662,41.92,-87.69
4,BE5E8EB4E7263A0B,41.900326,-87.696697,41.90,-87.70


### Compute Geohash on Divvy data

In [8]:
transform_lonlat_features(df_geo)

NameError: name 'transform_lonlat_features' is not defined

In [9]:
df_geo.head()

,ride_id,start_lat,start_lon,end_lat,end_lon
0,E19E6F1B8D4C42ED,41.900341,-87.696743,41.89,-87.72
1,DC88F20C2C55F27F,41.900333,-87.696707,41.90,-87.69
2,EC45C94683FE3F27,41.900313,-87.696643,41.90,-87.70
3,4FA453A75AE377DB,41.900399,-87.696662,41.92,-87.69
4,BE5E8EB4E7263A0B,41.900326,-87.696697,41.90,-87.70


In [10]:
compute_geohash_rides(df_geo)

,ride_id,start_lat,start_lon,end_lat,end_lon,geohash_start,geohash_end
0,E19E6F1B8D4C42ED,41.900341,-87.696743,41.890000,-87.720000,dp3wk,dp3w7
1,DC88F20C2C55F27F,41.900333,-87.696707,41.900000,-87.690000,dp3wk,dp3wk
2,EC45C94683FE3F27,41.900313,-87.696643,41.900000,-87.700000,dp3wk,dp3wk
3,4FA453A75AE377DB,41.900399,-87.696662,41.920000,-87.690000,dp3wk,dp3wk
4,BE5E8EB4E7263A0B,41.900326,-87.696697,41.900000,-87.700000,dp3wk,dp3wk
...,...,...,...,...,...,...,...
374947,9397BDD14798A1BA,41.900960,-87.623777,41.890847,-87.618617,dp3wq,dp3wq
374948,BBBEB8D51AAD40DA,41.889177,-87.638506,41.890847,-87.618617,dp3wm,dp3wq
374949,637FF754DA0BD9E1,41.900960,-87.623777,41.977997,-87.668047,dp3wq,dp3wv
374950,F8F43A0B978A7A35,41.889177,-87.638506,41.890847,-87.618617,dp3wm,dp3wq


### Import and format station data

In [11]:
# path_stations = os.path.
path = '/Users/joachimclodic/code/G-Dolle/DIVVY_BIKE/raw_data/Divvy_Stations_2014-Q3Q4.csv'
df_stations=pd.read_csv(path)
df_stations.head()

,id,name,latitude,longitude,dpcapacity,dateCreated
0,5,State St & Harrison St,41.873958,-87.627739,19,6/10/2013 10:46
1,13,Wilton Ave & Diversey Pkwy,41.932500,-87.652681,19,6/22/2013 18:29
2,14,Morgan St & 18th St,41.858086,-87.651073,15,6/22/2013 18:33
3,15,Racine Ave & 19th St,41.856453,-87.656471,15,6/22/2013 18:35
4,16,Wood St & North Ave,41.910329,-87.672516,15,6/22/2013 18:55


In [12]:
df_stations=df_stations.rename(columns={'latitude':'lat','longitude':'lon'})
df_stations.head()

,id,name,lat,lon,dpcapacity,dateCreated
0,5,State St & Harrison St,41.873958,-87.627739,19,6/10/2013 10:46
1,13,Wilton Ave & Diversey Pkwy,41.932500,-87.652681,19,6/22/2013 18:29
2,14,Morgan St & 18th St,41.858086,-87.651073,15,6/22/2013 18:33
3,15,Racine Ave & 19th St,41.856453,-87.656471,15,6/22/2013 18:35
4,16,Wood St & North Ave,41.910329,-87.672516,15,6/22/2013 18:55


### Compute Geohash on stations data

In [13]:
compute_geohash_stations(df_stations)

,id,name,lat,lon,dpcapacity,dateCreated,geohash
0,5,State St & Harrison St,41.873958,-87.627739,19,6/10/2013 10:46,dp3wj
1,13,Wilton Ave & Diversey Pkwy,41.932500,-87.652681,19,6/22/2013 18:29,dp3wt
2,14,Morgan St & 18th St,41.858086,-87.651073,15,6/22/2013 18:33,dp3wj
3,15,Racine Ave & 19th St,41.856453,-87.656471,15,6/22/2013 18:35,dp3wj
4,16,Wood St & North Ave,41.910329,-87.672516,15,6/22/2013 18:55,dp3wk
...,...,...,...,...,...,...,...
295,347,Ashland Ave & Grace St,41.950687,-87.668700,15,10/4/2013 16:06,dp3wt
296,348,California Ave & 21st St,41.854016,-87.695445,15,10/9/2013 16:13,dp3wh
297,349,Halsted St & Wrightwood Ave,41.929143,-87.649077,15,10/10/2013 12:40,dp3wt
298,350,Ashland Ave & Chicago Ave,41.895966,-87.667747,15,10/10/2013 14:22,dp3wm


In [14]:
df_stations_geohash=compute_geohash_stations(df_stations)
df_stations_geohash['geohash'].nunique()

14

In [49]:
df_stations_geohash['geohash'].unique()

array(['dp3wj', 'dp3wt', 'dp3wk', 'dp3wm', 'dp3wq', 'dp3wn', 'dp3ty',
       'dp3wh', 'dp3ws', 'dp3tz', 'dp3wv', 'dp3tv', 'dp3wu', 'dp3tw'],
      dtype=object)